# Import data

In [ ]:
pip install gdown

In [ ]:
pip install pulp==2.4

In [81]:
import pandas as pd
from pulp import *
import gdown

<IPython.core.display.Javascript object>

In [ ]:
# Install file from Google Drive

url = "https://drive.google.com/uc?id=15BPH7-3GGWBfXPJQ3stkT6SHQECbT-pt"
output = "shifts.csv"
gdown.download(url, output, quiet=False)

In [84]:
df = pd.read_csv("shifts.csv", index_col=0)
df

,Shift 1,Shift 2,Shift 3,Shift 4,Workers Required
Time Windows,,,,,
6:00 – 9:00,X,NaN,NaN,X,55.0
9:00 – 12:00,X,NaN,NaN,NaN,46.0
12:00 – 15:00,X,X,NaN,NaN,59.0
15:00 – 18:00,NaN,X,NaN,NaN,23.0
18:00 – 21:00,NaN,X,X,NaN,60.0
21:00 – 24:00,NaN,NaN,X,NaN,38.0
24:00 – 3:00,NaN,NaN,X,X,20.0
3:00 – 6:00,NaN,NaN,NaN,X,30.0
Wage rate per 9h shift ($),135,140,190,188,NaN


<IPython.core.display.Javascript object>

# Formulation 

## Problem statements
Input parameters:
   * Number of shifts ($n$)
   * Number of time windows ($T$)
   * Number of workers required per time window ($d_t$)
   * Wage rate per shift ($w_j$)
    
Decision variables: number of workers need per work shift ($y_j$)

Constraints: demand within each time window $t$ must be satisfied

Objective: Minimize the total cost of wages paid to all workers

Let $a_{jt}=1$ if shift $j$ covers the time window $t$ ($j$=1,..,n; t=1,...,$T$) and $a_{jt}=0$ otherwise

In [24]:
df = df.fillna(0).applymap(lambda x: 1 if x == "X" else x)

<IPython.core.display.Javascript object>

In [40]:
a = df.drop(index=["Wage rate per 9h shift ($)"], columns=["Workers Required"]).values
a

array([[1, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 0, 0, 1]], dtype=object)

<IPython.core.display.Javascript object>

In [60]:
# Input parameters
n = a.shape[1]
T = a.shape[0]
d = df["Workers Required"].values
w = df.loc["Wage rate per 9h shift ($)", :].values.astype(int)

<IPython.core.display.Javascript object>

In [70]:
# Create problem
prob = LpProblem("scheduling_workers", LpMinimize)

<IPython.core.display.Javascript object>

In [96]:
?LpVariable

<IPython.core.display.Javascript object>

In [71]:
# Decision variables
y = LpVariable.dicts("num_workers", list(range(n)), lowBound=0, cat="Integer")

<IPython.core.display.Javascript object>

## Formulation

The model formulation is:

\begin{align}
\text{Minimize} \;\; &\sum_j w_jy_j\\
\text{subject to} \;\; &\sum_j a_{jt}y_j \geq d_t \;\; t=1,..,T\\
& y_j \geq 0 \text{ and integer } j=1,...,n
\end{align}

In [72]:
# Objective
prob += lpSum([w[j] * y[j] for j in range(n)])

<IPython.core.display.Javascript object>

In [73]:
# Subject to
for t in range(T):
    prob += lpSum([a[t, j] * y[j] for j in range(n)]) >= d[t]

<IPython.core.display.Javascript object>

# Solve

In [74]:
prob.solve()

1

<IPython.core.display.Javascript object>

In [86]:
print("Status:", LpStatus[prob.status])

Status: Optimal


<IPython.core.display.Javascript object>

In [97]:
for shift in range(n):
    print(
        f"The number of workers needed for shift {shift} is {int(y[shift].value())} workers"
    )

The number of workers needed for shift 0 is 46 workers
The number of workers needed for shift 1 is 23 workers
The number of workers needed for shift 2 is 38 workers
The number of workers needed for shift 3 is 30 workers


<IPython.core.display.Javascript object>

In [91]:
print(
    f"The total amount of money being paid to workers in 4 shifts are ${value(prob.objective)}",
)

The total amount of money being paid to workers in 4 shifts are $22290.0


<IPython.core.display.Javascript object>